<a href="https://colab.research.google.com/github/npesudip/hello-world/blob/master/Assignment_Week_3_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IBM DS CAPSTONE - PART 1

### For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:


In [0]:
# Import necessary libraries

import requests
import lxml.html as lh
import pandas as pd

In [0]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

This means that there are 3 columns per row

In [4]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


#### Creating Pandas DataFrame

Each header is appended to a tuple along with an empty list.

In [0]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[287, 287, 287]

This shows that each of the 3 columns has exactly 288 rows

#### Creating the pandas data frame 

In [0]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

3.To create the above dataframe:

-  The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4.Submit a link to your Notebook on your Github repository. (10 marks)

- Note: There are different website scraping libraries and packages in Python. One of the most common packages is  BeautifulSoup. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

- The package is so popular that there is a plethora of tutorials and examples of how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

- Use the BeautifulSoup package or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe

In [8]:
# Access the top 5 rows of the data frame 
df.head()

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


#### Rearranging and renaming the columns

In [9]:
df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned\n,M1A,Not assigned
1,Not assigned\n,M2A,Not assigned
2,Parkwoods\n,M3A,North York
3,Victoria Village\n,M4A,North York
4,Harbourfront\n,M5A,Downtown Toronto


#### Cleaning the messy string in the Borough column

In [10]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto


#### Dropping all cells with a borough that is Not assigned

In [11]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Postcode,Borough,Neighbourhood
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto
5,Lawrence Heights,M6A,North York
6,Lawrence Manor,M6A,North York
7,Queen's Park,M7A,Downtown Toronto
8,Not assigned,M8A,Not assigned
9,Not assigned,M9A,Queen's Park


#### Combining Neighbourhoods based on similar Postcode and Borough  

In [12]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,Adelaide,M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough
2,Agincourt North,M1V,Scarborough
3,Albion Gardens,M9V,Etobicoke
4,Alderwood,M8W,Etobicoke
5,Bathurst Manor,M3H,North York
6,Bathurst Quay,M5V,Downtown Toronto
7,Bayview Village,M2K,North York
8,Beaumond Heights,M9V,Etobicoke
9,Bedford Park,M5M,North York


#### Removing any space in the start of the string

In [0]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

#### Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"

In [0]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [15]:
# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood


In [16]:
# Check the shape of the data frame
df.shape

(287, 3)

#### Save this file to a csv

In [0]:
df.to_csv(r'df_can.csv')

In [0]:
df.to_csv('data.csv')

In [25]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
df.to_csv('data.csv')
!cp data.csv "drive/My Drive/"

google drive file link
https://drive.google.com/open?id=1KcOqc_p8lRalrAUugIvgkV7O1WkvH95g

  #**IBM DS CAPSTONE - PART 2**


---



Extracting the Latitude and Longitude of Canada Ontario

In [19]:
"""import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')"""

"import requests # library to handle requests\nimport pandas as pd # library for data analsysis\nimport numpy as np # library to handle data in a vectorized manner\nimport random # library for random number generation\n\n!conda install -c conda-forge geopy --yes \nfrom geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values\n\n# libraries for displaying images\nfrom IPython.display import Image \nfrom IPython.core.display import HTML \n    \n# tranforming json file into a pandas dataframe library\nfrom pandas.io.json import json_normalize\n\n!conda install -c conda-forge folium=0.5.0 --yes\nimport folium # plotting library\n\nprint('Folium installed')\nprint('Libraries imported.')"

In [20]:
"""import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]"""

"import geocoder # import geocoder\n\n# initialize your variable to None\nlat_lng_coords = None\n\n# loop until you get the coordinates\nwhile(lat_lng_coords is None):\n  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n  lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]"

Couldn't figure out the error, So I used the csv given in the link


In [39]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

link = "http://cocl.us/Geospatial_data"
df1 = pd.read_csv(link)

df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
df1.shape

(103, 3)

In [40]:
df1.columns = ['Postcode','Latitude','Longitude']

cols = df1.columns.tolist()
cols

['Postcode', 'Latitude', 'Longitude']

Read in the CSV file saved in the previous assignment

In [43]:
url = 'df_can.csv'
df1 = pd.read_csv(url)
df.head()
# Dataset is now stored in a Pandas Dataframe

,Postcode,Borough,Neighbourhood
0,Adelaide,M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough
2,Agincourt North,M1V,Scarborough
3,Albion Gardens,M9V,Etobicoke
4,Alderwood,M8W,Etobicoke


Merging the two data frames together based on their Postcode

In [44]:
df_new = pd.merge(df, df1, on='Postcode')
df_new.head()

,Postcode,Borough_x,Neighbourhood_x,Unnamed: 0,Borough_y,Neighbourhood_y
0,Adelaide,M5H,Downtown Toronto,0,M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough,1,M1S,Scarborough
2,Agincourt North,M1V,Scarborough,2,M1V,Scarborough
3,Albion Gardens,M9V,Etobicoke,3,M9V,Etobicoke
4,Alderwood,M8W,Etobicoke,4,M8W,Etobicoke


In [0]:
df_new.to_csv(r'df_final.csv')

In [46]:
df_new.head()

,Postcode,Borough_x,Neighbourhood_x,Unnamed: 0,Borough_y,Neighbourhood_y
0,Adelaide,M5H,Downtown Toronto,0,M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough,1,M1S,Scarborough
2,Agincourt North,M1V,Scarborough,2,M1V,Scarborough
3,Albion Gardens,M9V,Etobicoke,3,M9V,Etobicoke
4,Alderwood,M8W,Etobicoke,4,M8W,Etobicoke


#**IBM DS CAPSTONE - PART 3**

In [54]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [0]:
toronto_df1= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

#boolean indexing        
toronto_df1 = toronto_df1.loc[toronto_df1['Borough']!="Not assigned"]
toronto_df2 = toronto_df1.groupby(['Postcode','Borough'],sort=False)['Neighbourhood'].agg(','.join).reset_index()
toronto_df2.loc[toronto_df2['Neighbourhood'] == "Not assigned", 'Neighbourhood'] = toronto_df2['Borough']

In [71]:
link = "df_final.csv"
neighborhoods = pd.read_csv(link,index_col=0)
neighborhoods.head()

,Postcode,Borough_x,Neighbourhood_x,Unnamed: 0.1,Borough_y,Neighbourhood_y
0,Adelaide,M5H,Downtown Toronto,0,M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough,1,M1S,Scarborough
2,Agincourt North,M1V,Scarborough,2,M1V,Scarborough
3,Albion Gardens,M9V,Etobicoke,3,M9V,Etobicoke
4,Alderwood,M8W,Etobicoke,4,M8W,Etobicoke


In [73]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()),neighborhoods.shape[0]
    )
)

KeyError: ignored

#### Find how many boroughs and neighborhoods


In [63]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [65]:
server.bind(("0.0.0.0", 6677))

NameError: ignored

In [66]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

KeyError: ignored